# **六、国盾量子计算云平台量子编程教学**

-----------------------

# 教学大纲
## 6. [其他常用功能函数](#jump_6)
### &nbsp;&nbsp;&nbsp;&nbsp; 6.1 [比特拓扑结构映射](#jump_6_1)
### &nbsp;&nbsp;&nbsp;&nbsp; 6.2 [线路正则检查(本地)](#jump_6_2)
### &nbsp;&nbsp;&nbsp;&nbsp; 6.3 [线路优化(本地)](#jump_6_3)
### &nbsp;&nbsp;&nbsp;&nbsp; 6.4 [语言互转：QCIS转QASM](#jump_6_4)
### &nbsp;&nbsp;&nbsp;&nbsp; 6.5 [语言互转：QASM转QCIS](#jump_6_5)
### &nbsp;&nbsp;&nbsp;&nbsp; 6.6 [其他说明](#jump_6_6)

# 6 其他常用功能函数<a id="jump_6"></a>

## 6.1 比特拓扑结构映射<a id="jump_6_1"></a>

超导量子处理器的比特不是全联通的，而是受到比特之间的拓扑连接限制，所以程序在执行时一定需要符合指定处理器的拓扑结构限制。  
而用户在开发程序时，如果不想考虑芯片的拓扑情况，则可以使用一些自动的比特映射程序来完成程序比特编号与物理比特编号的对应关系。  
需要注意的是，如果用户程序所使用的比特编号比较随意（如使尝试了全联通操作），则在映射程序的处理中未必能得到最优解，或者未必能得到有效的映射结果。如果为了满足映射需求，在映射过程中还有可能使用SWAP进行数据交互，则会耗费更多的门事件，从而占用了大量的比特寿命。甚至因为最终映射后的线路过长，导致程序无法执行或者程序结果全是噪声。  
因为映射程序设计到较多的算法，所以映射过程需要耗费一定时间，请耐心等待。

In [ ]:
qcis = '''
      X Q1
      X Q1
      Y Q4
      Z Q4
      H Q4
      S Q4
      SD Q5
      T Q6
      TD Q6
      X2P Q6
      X2M Q6
      CZ Q1 Q2
      Y2P Q1
      Y2M Q1
      CZ Q4 Q5
      CZ Q3 Q4
      CZ Q4 Q8
      CZ Q9 Q10
      RZ Q2 2.135648
      RX Q2 -1.135648
      RY Q3 1.8567
      RXY Q7 3.2 1.04
      CZ Q12 Q14
      I Q1 100
      B Q6 Q2
      M Q7
      M Q5
      M Q1 Q2
  '''
from ezQpy import *
account = Account(login_key='d69a8e1389f2cabde55c3de99fcf8589', machine_name='ClosedBetaQC')
qcis = account.qcis_mapping_isq(qcis)
#因为映射的效果和效率不同，我们与合作伙伴提供了多种映射方式，以上只为其中之一。
print(qcis)

## 6.2 线路正则检查(本地)<a id="jump_6_2"></a>

在用户客户端进行线路的正则检查，检查门和参数是否正确，但不检查芯片拓扑应用是否正确。  
当线路有代替换变量时，请不要执行正则检查和线路优化。

In [ ]:
#有代替换变量时，请不要执行正则检查和代码优化
qcis_raw=qcis
qcis_circuit=account.qcis_check_regular(qcis_raw)
print(qcis_circuit)

## 6.3 线路优化(本地)<a id="jump_6_3"></a>

在用户客户端进行线路的优化，尽可能将相邻的单比特门合并。该功能目前正在迭代，也希望大家反馈需求或共同参与社区开发。  
当线路有有代替换变量时，请不要执行正则检查和线路优化。

In [ ]:
#优化代码，对一些等效操作进行合并。现阶段提供的优化实例见SDK发布详细说明。https://xxxx
qcis_circuit = account.simplify(qcis_raw)
print(qcis_circuit)

## 6.4 语言互转：QCIS转QASM<a id="jump_6_4"></a>

为支持更多使用场景，我们提供了国内量子语言的支持，由开发商直接设计编译器，有其语言转为QCIS在云平台使用。  
同时，我们提供转化功能，将QCIS转为国际上的QASM语言。  
目前该函数未必是最优的，但是可以解决我们的基础需求。

In [ ]:
qcis = '''
      X Q1
      X Q1
      Y Q4
      Z Q4
      H Q4
      S Q4
      SD Q5
      T Q6
      TD Q6
      X2P Q6
      X2M Q6
      CZ Q1 Q2
      Y2P Q1
      Y2M Q1
      CZ Q4 Q5
      CZ Q3 Q4
      CZ Q4 Q8
      CZ Q9 Q10
      RZ Q2 2.135648
      RX Q2 -1.135648
      RY Q3 1.8567
      RXY Q7 3.2 1.04
      CZ Q12 Q14
      I Q1 100
      B Q6 Q2
      M Q7
      M Q5
      M Q1 Q2
  '''
from ezQpy import *
account = Account(login_key='d69a8e1389f2cabde55c3de99fcf8589', machine_name='ClosedBetaQC')
qasm=account.convert_qcis_to_qasm(qcis)
print(qasm)

## 6.5 语言互转：QASM转QCIS<a id="jump_6_5"></a>

因为现阶段的量子计算机的结果读取存在保真度的影响，所以在做统计结果时，应该根据实际情况进行结果修正，我们提供了一个示例程序。


In [ ]:
qcis=account.convert_qasm_to_qcis(qasm)
print(qcis)

## 6.6 其他说明<a id="jump_6_6"></a>

因云平台会因为任务压力过大，量子计算机响应异常，网络超时等原因，会造成一些实验步骤异常，如运行失败，线路长时间排队指定位置（非维护态）不变化等。  
所以在设计自动运行程序时，多对函数运行结果进行判断，或者用try异常捕获运行，对异常状态进行自动的处理，以保证任务能够长时间自动运行。